In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Latent(nn.Module):
    def __init__(self, init_log_sigma, const_sigma, **kwargs):
        super().__init__()
        self.log_sigma = nn.Parameter(torch.ones(1) * init_log_sigma, requires_grad=not const_sigma)
                
    def forward(self, data, **kwargs):
        # data['z'] : (N, c, H, W)
        # data['e'] : (M, c)
        
        z_dim = data['z'].shape[1]
        # (NHW, c)
        z = data['z'].permute(0, 2, 3, 1).reshape(-1, z_dim)
        N = len(z)
        T = kwargs['latent_temp'] if 'latent_temp' in kwargs else 1.0
        softmax_temp = kwargs['softmax_temp'] if 'softmax_temp' in kwargs else 1.0
        
        # (NHW, M) = sum((NHW, 1, z) - (1, M, z), dim=2)
        distance = torch.norm(z.unsqueeze(1) - data['e'].unsqueeze(0), dim=2) ** 2
        alpha = -1/(2*torch.exp(self.log_sigma)**2)
        matrix = alpha*distance/T
        data['matrix'] = matrix
        # (NHW, M)
        belong = data['belong'] if 'belong' in data else None
        loss = -torch.mean(T*CustomLogSumExp.apply(matrix, belong, softmax_temp))
        loss = loss + 0.5*z_dim*(2*self.log_sigma-np.log(np.e)) + np.log(N)        
        data['lse_loss'] = loss
        
        return data
    
class CustomLogSumExp(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, belong=None, temp=1):
        # input : (N, M)
        # belong : (N, M)
        
        ctx.temp = temp
        # (1, M)
        output = torch.logsumexp(input, dim=0, keepdim=True)
        ctx.save_for_backward(input, output, belong)
        return output.squeeze(0)  # output을 반환할 때는 차원을 줄입니다.

    @staticmethod
    def backward(ctx, grad_output):
        temp = ctx.temp
        # (N, M), (1, M), (N, M)
        input, output, belong = ctx.saved_tensors
        # softmax 함수를 사용하여 그래디언트 계산을 수행합니다.
        # (N, M)
        if belong is None:
            softmax_result = torch.exp(input - output)
        else:
            inner_value = belong * input + (1-belong) * (np.log(max(temp, 1e-15)) + input)
            softmax_result = torch.softmax(inner_value, dim=0)
        grad_input = softmax_result * grad_output.unsqueeze(0)
        return grad_input, None, None

In [67]:
latent = Latent(init_log_sigma=0, const_sigma=True)
data = {'z': torch.randn(4, 1, 2, 2),
        'e': torch.randn(10, 1),
        'belong': torch.randint(0, 2, size=(4*2*2, 10))}
latent(data)

{'z': tensor([[[[-1.3160, -0.2128],
           [-1.0209, -1.3800]]],
 
 
         [[[-0.7644,  0.7662],
           [ 0.5318,  1.7991]]],
 
 
         [[[ 0.2352,  0.9730],
           [-1.6667,  0.6466]]],
 
 
         [[[ 0.0054, -0.7841],
           [-2.0037, -0.5783]]]]),
 'e': tensor([[ 0.6101],
         [-1.0073],
         [ 0.5925],
         [-0.6866],
         [ 0.0954],
         [ 1.2205],
         [-1.6139],
         [ 0.0961],
         [ 0.9006],
         [ 0.3560]]),
 'belong': tensor([[1, 0, 0, 1, 1, 0, 1, 0, 0, 0],
         [0, 1, 1, 1, 1, 0, 0, 1, 1, 1],
         [1, 0, 1, 1, 1, 1, 1, 0, 1, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 1, 0, 1, 1, 0, 0, 1, 1],
         [1, 1, 1, 0, 1, 0, 1, 0, 0, 1],
         [0, 1, 1, 0, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 0, 1, 1, 1, 1, 1, 0],
         [1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
         [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
         [1, 0, 1, 1, 0, 0, 0, 1, 0, 1],
         [1, 0, 1, 1, 1, 0, 1, 0, 0, 0],
         [0, 0, 

In [68]:
# 함수를 사용하려면 apply 메서드를 사용하고, dim 매개변수를 전달합니다.
input = torch.randn(3, 4, requires_grad=True)
belong = torch.randint(0, 2, size=[*input.shape])
output = CustomLogSumExp.apply(input, belong, 0)
output.backward(torch.ones_like(output))
